### 필요한 모듈 포함하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### 데이터 전처리
데이터 모델이 데이터를 처리하기 쉬운 형태로 변환하기 위한 목적<br>
데이터셋 로드 전 데이터셋 전처리 방식 정의

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

### 데이터셋 준비
학습시키고자 하는 데이터셋 로드 및 전처리 적용<br>
학습이 목적인 `trainset`과 모델 정확도 테스트를 위한 `testset`으로 분류<br>
또한 CNN모델에 입력할 수 있도록 각 `trainset`과 `testset`을 `DataLoader`를 사용하여 변환

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=4, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=4, shuffle=False)

### CNN 모델 함수로 구현
Convolution 연산, Max Pool 기술을 사용하여 특징점을 추출하고,<br>
추출한 특징점을 가지고 이미지를 분류하는 기술

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() 
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

model = CNN()

### 모델 학습
데이터셋의 학습 데이터를 통해 모델을 학습시키는 함수

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

### 모델 테스트
학습된 모델의 성능을 평가하는 함수

In [ ]:
def test(model, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data in test_loader:
            data, target = data
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
        f'Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

### 평가방법
1. model에 데이터를 입력해 `예측값` 얻기
2. `예측값`과 `실제 레이블` 간 `손실 계산`
3.

### 모델 저장
학습된 모델을 저장하는 함수

In [ ]:
def save_model(model, epoch):
    model_path = os.path.join(model_dir, 'mnist_mlp_model_epoch_{}.pth'.format(epoch))
    torch.save(model.state_dict(), model_path)

### 모델 학습을 위한 준비 
1. 예측값과 실제값 사이 손실을 계산하는 함수 준비<br>
2. 모델 학습 과정에서 손실 함수 최소화를 위한 최적화 알고리즘 준비<br>
3. 훈련 횟수 준비

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs = 5

### 모델 학습 적용
훈련 횟수만큼 `훈련 > 테스트 > 모델 저장` 과정 반복<br>

In [ ]:
for epoch in range(epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader, epoch)
    save_model(mode, epoch)